In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import dependencies
import sqlalchemy
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
# Load the data
file_path = Path('./Resources/Clean_MC_Master.csv')
df = pd.read_csv(file_path)
df

,MLSNumber,Address,SoldPrice,CurrentPrice,ListDate,SettledDate,#ofStories,City,Zip Code,Subdivision,New Construction YN,Age,InteriorSqFt,Bedrooms,Baths,Garage YN,Structure Type
0,1002388281,9701 Fields Rd #1806,"$127,000","$129,900",11/9/2015,1/4/2016,Main,Gaithersburg,20878,WASHINGTON TOWER CODM,No,1966,446.0,0.0,1.0,No,Unit/Flat/Apartment
1,1002388133,2211 Washington Ave #W-102,"$202,000","$207,000",11/9/2015,1/4/2016,Main,Silver Spring,20910,ROCK CREEK APTS CODM 2,No,1948,671.0,1.0,1.0,No,Unit/Flat/Apartment
2,1002384775,3117 University Blvd W #B4,"$139,900","$139,900",10/28/2015,1/4/2016,Main,Kensington,20895,MONTGOMERY CENTURY,No,1973,754.0,1.0,1.0,No,Unit/Flat/Apartment
3,1002382327,10201 Grosvenor Pl #210,"$195,000","$199,900",10/15/2015,1/4/2016,Main,Rockville,20852,GROSVENOR PARK,No,1972,851.0,1.0,1.0,No,Unit/Flat/Apartment
4,1002382267,10301 Rossmore Ct,"$840,000","$850,000",10/22/2015,1/4/2016,"Lower1,Lower2,Main,Upper1",Bethesda,20814,WILDWOOD KNOLLS,No,1963,3060.0,4.0,4.0,Yes,Detached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61797,MDMC2005770,1 Paca Pl,"$625,000","$625,000",7/22/2021,9/16/2021,"Main,Upper1",Rockville,20852,HUNGERFORD,No,1955,2237.0,4.0,3.0,Yes,Detached
61798,MDMC753990,1108 Clagett Dr,"$499,500","$509,000",7/15/2021,9/16/2021,Main,Rockville,20851,ROCKCREST,No,1951,1457.0,3.0,3.0,Yes,Detached
61799,MDMC2003756,11307 Galt Ave,"$410,000","$445,000",7/26/2021,9/16/2021,"Lower1,Main,Upper1",Silver Spring,20902,WHEATON HILLS,No,1950,1872.0,4.0,2.0,No,Detached
61800,MDMC763464,8809 Thomas Lea Ter,"$400,000","$374,900",6/24/2021,9/16/2021,"Lower1,Main,Upper1",Montgomery Village,20886,THE REACH,No,1986,2160.0,4.0,4.0,No,Interior Row/Townhouse


In [4]:
# Drop MLS and Address columns
housing_df = df.drop(['MLSNumber', 'Zip Code', 'Address', 'ListDate', 'SettledDate', 'Subdivision', 'City', 'CurrentPrice'], axis=1)
housing_df.head()

,SoldPrice,#ofStories,New Construction YN,Age,InteriorSqFt,Bedrooms,Baths,Garage YN,Structure Type
0,"$127,000",Main,No,1966,446.0,0.0,1.0,No,Unit/Flat/Apartment
1,"$202,000",Main,No,1948,671.0,1.0,1.0,No,Unit/Flat/Apartment
2,"$139,900",Main,No,1973,754.0,1.0,1.0,No,Unit/Flat/Apartment
3,"$195,000",Main,No,1972,851.0,1.0,1.0,No,Unit/Flat/Apartment
4,"$840,000","Lower1,Lower2,Main,Upper1",No,1963,3060.0,4.0,4.0,Yes,Detached


In [5]:
# Convert SoldPrice to numerical
housing_df['SoldPrice'] = housing_df['SoldPrice'].str.replace(',', '').str.replace('$', '').astype(int)
housing_df.head()

,SoldPrice,#ofStories,New Construction YN,Age,InteriorSqFt,Bedrooms,Baths,Garage YN,Structure Type
0,127000,Main,No,1966,446.0,0.0,1.0,No,Unit/Flat/Apartment
1,202000,Main,No,1948,671.0,1.0,1.0,No,Unit/Flat/Apartment
2,139900,Main,No,1973,754.0,1.0,1.0,No,Unit/Flat/Apartment
3,195000,Main,No,1972,851.0,1.0,1.0,No,Unit/Flat/Apartment
4,840000,"Lower1,Lower2,Main,Upper1",No,1963,3060.0,4.0,4.0,Yes,Detached


In [6]:
# Convert text to numbers
clean_housing_df = pd.get_dummies(housing_df, drop_first = True)
clean_housing_df.head()

,SoldPrice,Age,InteriorSqFt,Bedrooms,Baths,"#ofStories_Lower1,Lower2","#ofStories_Lower1,Lower2,Lower3,Main","#ofStories_Lower1,Lower2,Lower3,Main,Upper1","#ofStories_Lower1,Lower2,Lower3,Main,Upper1,Upper2","#ofStories_Lower1,Lower2,Lower3,Upper1",...,#ofStories_Upper2,New Construction YN_Yes,Garage YN_Yes,Structure Type _End of Row/Townhouse,Structure Type _Garage/Parking Space,Structure Type _Interior Row/Townhouse,Structure Type _Other,Structure Type _Penthouse Unit/Flat/Apartment,Structure Type _Twin/Semi-Detached,Structure Type _Unit/Flat/Apartment
0,127000,1966,446.0,0.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,202000,1948,671.0,1.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,139900,1973,754.0,1.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,195000,1972,851.0,1.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,840000,1963,3060.0,4.0,4.0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
# Create features
X = clean_housing_df.drop('SoldPrice', axis=1)

# Create target
y = clean_housing_df['SoldPrice']

In [8]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=100)

In [9]:
# Instantiate a linear regression model and fit on training dataset.
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression()

In [10]:
# Make predictions on testing data.
y_pred = lin_model.predict(X_test)

In [11]:
# Print RMSE and accuracy.

# The mean squared error
print (f'Mean squared error: {mean_squared_error(y_test, y_pred):.2f}')

# Model accuracy:
print (f'Training accuracy: {lin_model.score(X_train, y_train):.2f}')
print (f'Testing accuracy: {lin_model.score(X_test, y_test):.2f}')

Mean squared error: 86846174669.31
Training accuracy: 0.59
Testing accuracy: 0.56


In [12]:
# Create a random forest regressor.
rf_model = RandomForestRegressor(n_estimators=100, random_state=50) 

In [13]:
# Fitting the model
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=50)

In [14]:
# Making predictions using the testing data.
y_pred = rf_model.predict(X_test)

In [15]:
# Display results
# The mean squared error
print (f'Mean squared error: {mean_squared_error(y_test, y_pred):.2f}')

# Model accuracy:
print (f'Training accuracy: {rf_model.score(X_train, y_train):.2f}')
print (f'Testing accuracy: {rf_model.score(X_test, y_test):.2f}')

Mean squared error: 56137953544.35
Training accuracy: 0.96
Testing accuracy: 0.71


In [16]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([1.34462157e-01, 3.10385953e-01, 2.66781171e-02, 4.51122913e-01,
       2.48001898e-09, 5.29061350e-06, 2.41264226e-05, 1.03551512e-05,
       4.04935636e-07, 1.57924855e-04, 1.21785445e-03, 1.16492485e-03,
       1.88864636e-05, 1.19940246e-06, 6.47138038e-05, 1.11570820e-05,
       1.30408960e-06, 2.97926032e-03, 6.20997325e-03, 6.83231680e-03,
       2.30318078e-04, 1.09277300e-05, 2.41097418e-04, 3.00487677e-07,
       1.19578030e-06, 3.03596693e-04, 4.41346042e-05, 1.65678097e-06,
       3.55332194e-06, 1.52373310e-07, 5.23751193e-06, 3.73363638e-05,
       1.57492624e-06, 1.70811475e-05, 3.58520803e-06, 1.26062043e-06,
       7.86631293e-08, 7.06914795e-03, 2.69726611e-03, 3.45500275e-03,
       1.97949192e-04, 2.82879123e-05, 3.92634696e-05, 6.84419488e-07,
       7.34322578e-05, 1.07541934e-04, 1.04457555e-05, 1.05900554e-07,
       1.18155380e-05, 6.17593536e-03, 9.65147942e-03, 1.94711477e-03,
       7.11922442e-06, 3.06242474e-03, 1.22313509e-03, 2.89174855e-03,
      

In [19]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.45112291294745266, 'Baths'),
 (0.3103859531261533, 'InteriorSqFt'),
 (0.13446215702857905, 'Age'),
 (0.02667811706975615, 'Bedrooms'),
 (0.018875917493643198, 'Structure Type _Unit/Flat/Apartment'),
 (0.009651479423137462, 'Garage YN_Yes'),
 (0.007069147948909399, '#ofStories_Main'),
 (0.006832316801730835, '#ofStories_Lower1,Main,Upper1,Upper2'),
 (0.006209973254971584, '#ofStories_Lower1,Main,Upper1'),
 (0.00617593535695813, 'New Construction YN_Yes'),
 (0.0034550027471779462, '#ofStories_Main,Upper1,Upper2'),
 (0.003062424739816082, 'Structure Type _Interior Row/Townhouse'),
 (0.002979260320699376, '#ofStories_Lower1,Main'),
 (0.002891748554397353, 'Structure Type _Penthouse Unit/Flat/Apartment'),
 (0.002697266109456031, '#ofStories_Main,Upper1'),
 (0.0019471147657023952, 'Structure Type _End of Row/Townhouse'),
 (0.0012231350921136175, 'Structure Type _Other'),
 (0.0012178544525699463, '#ofStories_Lower1,Lower2,Main,Upper1'),
 (0.0011649248461171049, '#ofStories_Lower1,Lower2,M